<a href="https://colab.research.google.com/github/Taichi2005/GoogleColab_Whisper/blob/main/%E3%80%90%E5%AE%8C%E6%88%90%E7%89%88%E3%80%91%E9%AB%98%E6%80%A7%E8%83%BD%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97%E5%AE%9F%E8%A1%8C%E3%82%BB%E3%83%AB_(%E3%83%A2%E3%83%87%E3%83%AB%E3%83%BB%E9%87%8F%E5%AD%90%E5%8C%96_%E9%81%B8%E6%8A%9E%E8%82%A2%E8%BF%BD%E5%8A%A0%E7%89%88).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# セル1: 環境構築
# -------------------------------------------------------------------------------------------
# GPUが有効になっているかを確認
print("▼ GPUの確認")
!nvidia-smi

# faster-whisperと、GPUで動作させるために必要なCUDAライブラリをインストール
print("\n▼ 必要なライブラリのインストール")
!pip install faster-whisper==1.0.3 -q
!pip install nvidia-cublas-cu12==12.6.4.1 -q
!pip install nvidia-cudnn-cu12==9.10.2.21 -q

print("\n✅ 環境構築が完了しました。")

In [ ]:
# -------------------------------------------------------------------------------------------
# セル2: Google Driveへの接続
# -------------------------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
print("\n✅ Google Driveへの接続が完了しました。")

In [ ]:
#@title 🚀 高性能文字起こし実行セル (モデル・量子化 選択肢追加版)
#@markdown ### 1. Google Driveのパス設定
#@markdown ---
#@markdown 動画・音声ファイルが保存されているフォルダと、結果を保存するフォルダのパスを指定します。
drive_audio_input_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/input_audio" #@param {type:"string"}
drive_transcript_output_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts" #@param {type:"string"}

#【変更点】モデルの選択肢をレポートに基づき追加
#@markdown ### 2. モデルとパフォーマンス設定
#@markdown ---
#@markdown **モデル**: `Zoont/...-int8-ct2`は精度を維持しつつ最速・省メモリな**総合推奨モデル**です。<br>
#@markdown **計算タイプ**: `int8`モデルには`int8_float16`、`float16`モデルには`float16`の組み合わせを推奨します。
model_name = "Zoont/faster-whisper-large-v3-turbo-int8-ct2" #@param ["Zoont/faster-whisper-large-v3-turbo-int8-ct2", "deepdml/faster-whisper-large-v3-turbo-ct2", "large-v3", "large-v2", "distil-large-v3", "medium", "small", "base", "tiny"]
#【変更点】量子化（計算タイプ）の選択肢をレポートに基づき追加
compute_type = "int8_float16" #@param ["int8_float16", "float16", "int8", "float32"]

#@markdown ### 3. VAD (音声区間検出) 設定
#@markdown ---
#@markdown VADを有効にすると、無音区間を自動で除去し、文字起こしの精度を向上させることができます。
use_vad_filter = True #@param {type:"boolean"}
#@markdown ここで指定したミリ秒以上の無音を「発話の区切り」とみなします。
vad_min_silence_duration_ms = 500 #@param {type:"slider", min:100, max:2000, step:100}

#@markdown ### 4. 高度な設定（オプション）
#@markdown ---
#@markdown **ビームサイズ (beam_size)**: 文字起こしの精度と速度のトレードオフを調整します。<br>
#@markdown 値を大きくすると精度が向上する可能性がありますが、処理速度は低下します。`faster-whisper`では`5`がバランスの取れた推奨値です。
beam_size = 5 #@param {type:"slider", min:1, max:10, step:1}


# --- ここから下は実行コード（編集不要） ---
import os
import glob
import shutil
from faster_whisper import WhisperModel
import time
import subprocess
from datetime import datetime, timezone, timedelta
from tqdm.notebook import tqdm

def get_current_timestamp():
    """現在時刻を[YYYY-MM-DD HH:MM:SS]形式の文字列で返す"""
    JST = timezone(timedelta(hours=+9))
    return datetime.now(JST).strftime('%Y-%m-%d %H:%M:%S')

def preprocess_media_file(drive_path, temp_dir):
    """
    メディアファイル（動画・音声）を前処理する関数。
    動画の場合はFFmpegで音声を抽出し、一時音声ファイルのパスを返す。
    音声の場合はそのままローカルにコピーし、そのパスを返す。
    """
    filename = os.path.basename(drive_path)
    base_filename, file_extension = os.path.splitext(filename)

    local_media_path = os.path.join(temp_dir, filename)
    shutil.copy(drive_path, local_media_path)

    supported_video_formats = ['.mp4', '.mov', '.avi', '.wmv', '.mkv', '.flv', '.webm']
    if file_extension.lower() in supported_video_formats:
        print(f"  - 動画ファイルを検出。音声の抽出を開始...")
        temp_audio_file = os.path.join(temp_dir, f"extracted_audio_{base_filename}_{int(time.time()*1000)}.wav")
        try:
            subprocess.run(
                ['ffmpeg', '-i', local_media_path, '-vn', '-ar', '16000', '-ac', '1', '-y', '-loglevel', 'error', temp_audio_file],
                check=True
            )
            os.remove(local_media_path)
            print(f"  - 音声の抽出が完了 -> {os.path.basename(temp_audio_file)}")
            return temp_audio_file
        except subprocess.CalledProcessError as e:
            print(f"  - 💥 FFmpegエラー ({filename}): 音声の抽出に失敗しました。 {e}")
            if os.path.exists(local_media_path): os.remove(local_media_path)
            return None
    else:
        return local_media_path

# 1. 環境設定とディレクトリの準備
print(f"{get_current_timestamp()} --- 1. 環境設定とディレクトリの準備 ---")
os.makedirs(drive_audio_input_dir, exist_ok=True)
os.makedirs(drive_transcript_output_dir, exist_ok=True)
print(f"入力フォルダ: {drive_audio_input_dir}")
print(f"出力フォルダ: {drive_transcript_output_dir}")

# 2. モデルのロード
print(f"\n{get_current_timestamp()} --- 2. モデル '{model_name}' ({compute_type}) をロード中... ---")
start_load_time = time.time()
model = WhisperModel(model_name, device="cuda", compute_type=compute_type)
end_load_time = time.time()
print(f"✅ モデルのロードが完了しました。({end_load_time - start_load_time:.2f}秒)")

# 3. 処理対象ファイルの検索
print(f"\n{get_current_timestamp()} --- 3. 処理対象ファイルの検索 ---")
supported_formats = ['*.mp3', '*.wav', '*.m4a', '*.flac', '*.ogg', '*.opus',
                     '*.mp4', '*.mov', '*.avi', '*.wmv', '*.mkv', '*.flv', '*.webm']
media_files = []
for fmt in supported_formats:
    media_files.extend(glob.glob(os.path.join(drive_audio_input_dir, fmt)))

if not media_files:
    print("⚠️ 入力フォルダに処理対象のメディアファイルが見つかりませんでした。")
else:
    print(f"✅ {len(media_files)} 件のメディアファイルを検出しました。")
    # 4. メイン処理ループ
    print(f"\n{get_current_timestamp()} --- 4. 文字起こし処理開始 ---")
    for media_path_drive in tqdm(media_files, desc="全体進捗"):
        start_process_time = time.time()
        filename = os.path.basename(media_path_drive)
        print(f"\n{get_current_timestamp()} ■ 処理開始: {filename}")

        local_audio_path = None
        try:
            local_audio_path = preprocess_media_file(media_path_drive, temp_dir='/content/')

            if local_audio_path is None:
                print(f"  - ファイル処理をスキップします。")
                continue

            # 文字起こし実行
            print(f"  - 文字起こしを実行中... (beam_size: {beam_size}, VAD: {'有効' if use_vad_filter else '無効'})")
            segments, info = model.transcribe(
                local_audio_path,
                beam_size=beam_size,
                vad_filter=use_vad_filter,
                vad_parameters=dict(min_silence_duration_ms=vad_min_silence_duration_ms)
            )
            print(f"  - 検出言語: {info.language} (確率: {info.language_probability:.2f})")

            base_filename, _ = os.path.splitext(filename)
            output_txt_filename = f"{base_filename}.txt"
            local_output_path = os.path.join('/content/', output_txt_filename)
            drive_output_path = os.path.join(drive_transcript_output_dir, output_txt_filename)

            # --- ここから変更 ---
            # 処理日時の情報を準備
            JST = timezone(timedelta(hours=+9))
            now = datetime.now(JST)
            timestamp_str = now.strftime('%Y-%m-%d %H:%M:%S')
            weekdays_jp = ["月", "火", "水", "木", "金", "土", "日"]
            weekday_str = weekdays_jp[now.weekday()]

            with open(local_output_path, "w", encoding="utf-8") as f:
                # メタデータ（ヘッダー情報）を書き込む
                f.write(f"元ファイル名: {filename}\n")
                f.write(f"処理完了日時: {timestamp_str} ({weekday_str}曜日)\n")
                f.write(f"使用モデル: {model_name}\n")
                f.write(f"計算タイプ: {compute_type}\n")
                f.write(f"VADフィルター: {'有効' if use_vad_filter else '無効'}\n\n")
                f.write("---\n\n")

                # 文字起こし結果を書き込む
                for segment in segments:
                    line = f"[{segment.start:0>7.2f}s -> {segment.end:0>7.2f}s] {segment.text.strip()}\n"
                    f.write(line)

            shutil.move(local_output_path, drive_output_path)
            print(f"  - 結果を保存しました: {drive_output_path}")

        except Exception as e:
            print(f"  - 💥 不明なエラーが発生しました: {e}")

        finally:
            if local_audio_path and os.path.exists(local_audio_path):
                os.remove(local_audio_path)
            end_process_time = time.time()
            print(f"{get_current_timestamp()} ■ 処理完了 ({end_process_time - start_process_time:.2f}秒)")

    print(f"\n{get_current_timestamp()} --- 🎉 全てのファイルの処理が完了しました ---")